In [30]:
import hopsworks
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import os
import joblib
import util
import time

dir="prophet"
weather=False

project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/16503
Connected. Call `.close()` to terminate connection gracefully.


In [17]:
mr = project.get_model_registry()
model_reg = mr.get_models("prophet")[-1]
model_dir = model_reg.download()
model = joblib.load(model_dir + "/prophet.pkl")

Connected. Call `.close()` to terminate connection gracefully.


['prophet/prophet.pkl']

In [20]:
%%writefile predict_example.py
import os
import numpy as np
import joblib
import hsfs

class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""        
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/prophet.pkl")
        print("Initialization Complete")

    def predict(self, inputs):
        """ Serves a prediction request with a trained model"""
        
        future = self.model.make_future_dataframe(periods=2500, freq='1min', include_history = False)
        forecast = self.model.predict(future)

        return forecast.to_numpy().tolist()

Writing predict_example.py


In [21]:
# Get the dataset API for the current project
dataset_api = project.get_dataset_api()

# Specify the local file path of the Python script to be uploaded
local_script_path = "predict_example.py"

# Upload the Python script to the "Models", and overwrite if it already exists
uploaded_file_path = dataset_api.upload(local_script_path, "Models/" + dir + f"/{model_reg.version}", overwrite=True)

# Create the full path to the uploaded script for future reference
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)


Uploading: 0.000%|          | 0/604 elapsed<00:00 remaining<?

In [25]:
# Deploy the fraud model
deployment = model_reg.deploy(
    name="mensa",  # Specify a name for the deployment
    script_file=predictor_script_path,  # Provide the path to the Python script for prediction
)

# Print the name of the deployment
print("Deployment: " + deployment.name)

# Display information about the deployment
deployment.describe()

time.sleep(45)

Deployment created, explore it at https://snurran.hops.works/p/16503/deployments/11265
Before making predictions, start the deployment by using `.start()`
Deployment: mensa
{
    "artifact_version": 1,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2024-08-16T11:54:31.939Z",
    "creator": "Jim Dowling",
    "description": null,
    "id": 11265,
    "inference_logging": "NONE",
    "model_framework": "PYTHON",
    "model_name": "prophet",
    "model_path": "/Projects/schwartz/Models/prophet",
    "model_server": "PYTHON",
    "model_version": 3,
    "name": "mensa",
    "predictor": "predict_example.py",
    "predictor_resources": {
        "limits": {
            "cores": 1.0,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 0.2,
            "gpus": 0,
            "memory": 32
        }
    },
    "requested_instances": 0,
    "serving_tool": "KSERVE"
}


In [26]:
deployment.start(await_running=300)
deployment.get_state().describe()


  0%|          | 0/5 [00:00<?, ?it/s]

Start making predictions by using `.predict()`
{
    "available_instances": 1,
    "available_transformer_instances": 0,
    "condition": {
        "reason": "Deployment is ready",
        "status": true,
        "type": "READY"
    },
    "deployed": "2024-08-16T12:25:27.000Z",
    "hopsworks_inference_path": "/project/16503/inference/models/mensa",
    "model_server_inference_path": "/v1/models/mensa",
    "revision": "04149134",
    "status": "Running"
}
